In [1]:
import pandas as pd
import numpy as np
import glob
import os

import multiprocessing
from multiprocessing import Pool

import pickle

num_cores = multiprocessing.cpu_count()

In [2]:
ITEMS_TO_TAKE = 100

In [6]:
# base_path = '/kaggle'
feature_dir = f"./input/indoorunifiedwifids"
train_files = sorted(glob.glob(os.path.join(feature_dir, '*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_dir, '*_test.csv')))

In [7]:
def generate_training_data(train_file):
    train = pd.read_csv(train_file)

    file_name = train_file.split('/')[-1]

    num_of_lines = train.shape[0]
    print(f'{file_name} : {num_of_lines}')

    data = []
    for i in range(num_of_lines):
        if i % 100 == 0:
            print(f'current:{i}')

        tmp = train.iloc[i,1:-4].astype(int).sort_values(ascending=False).head(ITEMS_TO_TAKE)
        target = train.iloc[i,-4:]
        line = [*tmp.index.astype(str), *tmp.values, *target]
        data.append(line)
    data = pd.DataFrame(data)
    data.columns = [f'bssid_{str(i)}' for i in range(ITEMS_TO_TAKE)] + [f'rssi_{str(i)}' for i in range(ITEMS_TO_TAKE)] + ['x','y','floor','path']
    data.to_csv(f'{file_name}', index=False)

In [9]:
with Pool(num_cores) as pool:
    pool.map(generate_training_data, [t for t in train_files])  

KeyboardInterrupt: 

In [ ]:
def generate_test_data(test_file):
    train = pd.read_csv(test_file)

    file_name = test_file.split('/')[-1]

    num_of_lines = train.shape[0]
    print(f'{file_name} : {num_of_lines}')

    data = []
    for i in range(num_of_lines):
        if i % 100 == 0:
            print(f'current:{i}')

        tmp = train.iloc[i,1:-4].astype(int).sort_values(ascending=False).head(ITEMS_TO_TAKE)
        target = train.iloc[i, [-1]]

        line = [*tmp.index.astype(str), *tmp.values, *target]
        data.append(line)
    data = pd.DataFrame(data)
    data.columns = [f'bssid_{str(i)}' for i in range(ITEMS_TO_TAKE)] + [f'rssi_{str(i)}' for i in range(ITEMS_TO_TAKE)] + ['site_path_timestamp']
    data.to_csv(f'{file_name}', index=False)

In [ ]:
with Pool(num_cores) as pool:
    pool.map(generate_test_data, [t for t in test_files])  

In [ ]:
def combine_all_files(files):
    for n_files, file in enumerate(files):
        file = file.replace('\\', '/')
        print(file)
        if data is None:
            data = pd.read_csv(file)
        else:
            wifi_file = np.loadtxt(file, delimiter=',', dtype='str')

            wifi_file = pd.DataFrame(wifi_file)
            wifi_file = wifi_file.iloc[1:,:]
            wifi_file.columns = [f'bssid_{i}' for i in range(100)] + [f'rssi_{i}' for i in range(100)] + ['x', 'y', 'floor', 'path']
            for col in RSSI_FEATS + ['floor']:
                wifi_file[col] = wifi_file[col].astype('int')
            wifi_file['x'] = wifi_file['x'].astype('float')
            wifi_file['y'] = wifi_file['y'].astype('float')

            data = pd.concat([data, wifi_file])
            data.reset_index(drop=True, inplace=True)
    return data

In [ ]:
pickle.dump(combine_all_files(train_files), open("train_all.pkl", 'wb'))

pickle.dump(combine_all_files(train_files), open("test_all.pkl", 'wb'))